In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path #ByteLevelBPETokenizer
from tokenizers.implementations import BertWordPieceTokenizer
from tokenizers.processors import BertProcessing
from transformers import BertConfig, BertForPreTraining, LineByLineTextDataset, BertTokenizerFast, BertTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch
torch.cuda.is_available()

True

In [2]:
root = 'HateSpeech/'
os.listdir('HateSpeech/')

['benchmarking', 'notebooks', 'data']

In [4]:
all_aae = pd.read_csv(f'{root}/data/aae_all.csv')
all_aae

,tweet_ids,timestamps,user_ids,long_latitudes,census_blockgroups,texts,demographic_inferences_0,demographic_inferences_1,demographic_inferences_2,demographic_inferences_3,cleaned_text
0,288532237186646000,Tue Jan 08 06:27:00 +0000 2013,1040150983,"[-87.322161, 41.6006888]",1.808901e+11,Man imissed a called from my bae hella mad -_-...,0.942000,0.058000,0.000000,0.000000,Man imissed a called from my bae hella mad -_-
1,289899995472224000,Sat Jan 12 01:01:59 +0000 2013,1068611971,"[-78.85113963, 42.909513]",3.602900e+11,@devontekthomas OMG I keep sayin boo wen I mea...,0.951111,0.031111,0.000000,0.017778,OMG I keep sayin boo wen I mean boy
2,289901379869351000,Sat Jan 12 01:07:29 +0000 2013,1068611971,"[-78.8510475, 42.90955088]",3.602900e+11,@devontekthomas I did not mean to say dat,0.957143,0.034286,0.000000,0.008571,I did not mean to say dat
3,289902453367910000,Sat Jan 12 01:11:45 +0000 2013,1068611971,"[-78.85107559, 42.90955579]",3.602900e+11,@devontekthomas awww u do too,0.975000,0.010000,0.000000,0.015000,awww u do too
4,290208633545965000,Sat Jan 12 21:28:24 +0000 2013,1068611971,"[-78.85091984, 42.90915702]",3.602900e+11,@devontekthomas wat r u doin boy,0.996000,0.000000,0.000000,0.004000,wat r u doin boy
...,...,...,...,...,...,...,...,...,...,...,...
1045461,413574589852045000,Thu Dec 19 07:40:40 +0000 2013,1099719194,"[-87.59259097, 40.1438471]",1.718300e+11,Thinkin about a lot. \u21e6\u21e6\u21e6\u21e6,0.833333,0.026667,0.003333,0.136667,Thinkin about a lot. \u21e6\u21e6\u21e6\u21e6
1045462,413575055541415000,Thu Dec 19 07:42:31 +0000 2013,1099719194,"[-87.59259097, 40.1438471]",1.718300e+11,then Pandora \u266c\u266a got me all n my feelin,0.924000,0.014000,0.006000,0.056000,then Pandora \u266c\u266a got me all n my feelin
1045463,413600606851330000,Thu Dec 19 09:24:03 +0000 2013,1099719194,"[-87.59262424, 40.14380614]",1.718300e+11,Oomf funny,0.960000,0.020000,0.000000,0.020000,Oomf funny
1045464,411550720647630000,Fri Dec 13 17:38:32 +0000 2013,1109521242,"[-84.12712217, 33.72779205]",1.308902e+11,Lmfaoo I see tae \ud83d\ude02\ud83d\ude02\ud83...,0.936000,0.040000,0.008000,0.016000,Lmfaoo I see tae \ud83d\ude02\ud83d\ude02\ud83...


In [5]:
no_ws = all_aae['cleaned_text'].replace("\n"," ")
no_ws = no_ws.str.strip()
clean_texts = no_ws.values.astype(str)
no_ws.to_csv(f'{root}data/temp_aae_cleaned.txt',index=None, header=False)
no_ws

In [16]:
slow_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

TypeError: from_pretrained() missing 1 required positional argument: 'pretrained_model_name_or_path'

In [31]:
save_path = f"{root}/benchmarking/models/bert_base_uncased/"
bert_vocab = f'{root}/benchmarking/models/bert_base_uncased/bert-base-uncased-vocab.txt'

In [32]:
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer(
    bert_vocab,
    lowercase=True,
    strip_accents=True,
)

In [33]:
# tokenizer = BertWordPieceTokenizer(
#     "bert-base-uncased",
#     strip_accents=True,
#     lowercase=True,
# )

tokenizer.train(
    f'{root}data/temp_aae_cleaned.txt',
    min_frequency=2,
    # special_tokens=[
    #     "<s>",
    #     "<pad>",
    #     "</s>",
    #     "<unk>",
    #     "<mask>",
    # ],
)

In [34]:
tokenizer.save(f'{root}benchmarking/models/aae_model/temp.json') #,'aae')

In [35]:
# tokenizer.enable_truncation(max_length=128) # todo change to max seq length
encode_example = tokenizer.encode(clean_texts[0])
print(
    encode_example.tokens
)

['[CLS]', 'man', 'imissed', 'a', 'called', 'from', 'my', 'bae', 'hella', 'mad', '-', '_', '-', '[SEP]']


In [3]:
trans_tokenizer = BertTokenizerFast.from_pretrained(f'{root}benchmarking/models/aae_model/', max_len=512)

In [4]:
dataset = LineByLineTextDataset(
    file_path=f'{root}data/temp_aae_cleaned.txt',
    tokenizer=trans_tokenizer,
    block_size=128
)

/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [39]:
dataset.examples

[{'input_ids': tensor([    2,   284, 21822,    41,  1121,   450,   129,  1022,  1755,   541,
             17,    39,    17,     3])},
 {'input_ids': tensor([   2, 1127,   49,  513, 2223,  636,  776,   49,  651,  501,    3])},
 {'input_ids': tensor([  2,  49, 396, 227, 651, 119, 331, 415,   3])},
 {'input_ids': tensor([   2, 2300,   61,  190,  308,    3])},
 {'input_ids': tensor([   2,  294,   58,   61, 1297,  501,    3])},
 {'input_ids': tensor([   2,   49,  264, 1314, 1388,    3])},
 {'input_ids': tensor([   2,   49,   11,   53,  905,  119,  171, 9614,   18,   18,   18,    3])},
 {'input_ids': tensor([   2, 4008,   18,   18,   18,   18,   18,   18, 6858,   18,   18,   18,
            18,   18,   18, 3466,   18,   18,   18,   18,   18,   18,   18,   18,
            18,   18,   18,  733,  349, 1352,   41, 1478, 4609,  119, 3852,  346,
          1263,   18,   18,   18,   18,   18,   18,   18,   18,   18,   18,   18,
            18,    7, 7965,    3])},
 {'input_ids': tensor([    2,   282

In [5]:

data_collator = DataCollatorForLanguageModeling(
    tokenizer=trans_tokenizer, mlm=True, mlm_probability=0.15
)

In [6]:
config = BertConfig(
    vocab_size=50000,
    max_length=512,
)
model = BertForPreTraining.from_pretrained('bert-base-uncased')

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
bert_aae_path = f'{root}benchmarking/models/aae_bert_lm/'
if not os.path.exists(bert_aae_path):
    os.makedirs(bert_aae_path)

training_args = TrainingArguments(
    output_dir=bert_aae_path,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/transformers/models/bert/modeling_bert.py", line 1069, in forward
    outputs = self.bert(
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/transformers/models/bert/modeling_bert.py", line 971, in forward
    encoder_outputs = self.encoder(
  File "/nethome/mhalevy3/anaconda

In [9]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/transformers/models/bert/modeling_bert.py", line 1069, in forward
    outputs = self.bert(
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/nethome/mhalevy3/anaconda3/envs/howard/lib/python3.9/site-packages/transformers/models/bert/modeling_bert.py", line 971, in forward
    encoder_outputs = self.encoder(
  File "/nethome/mhalevy3/anaconda

In [ ]:
# http://slanglab.cs.umass.edu/TwitterAAE/
columns = ['tweet_ids', 'timestamps', 'user_ids', 'long_latitudes', 'census_blockgroups', 'texts', 'demographic_inferences_0','demographic_inferences_1','demographic_inferences_2','demographic_inferences_3']
# dtypes = [str, pd.datetime, str, (float, float), float, str, (float, float, float, float)]
all_aae = pd.read_csv(f'{root}data/twitteraae_all_aa.csv', header=0, names=columns)
print(all_aae.shape)
all_aae.dropna(inplace=True)

In [ ]:
# clean texts
def preprocess_twitter_davidson(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned

    # TODO this is taken from Davidson 2017, consider removing !!!!!! on retweets and extra :
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
                       '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    # parsed_text = parsed_text.code("utf-8", errors='ignore')
    return parsed_text

all_aae['cleaned_text'] = all_aae['texts'].apply(preprocess_twitter_davidson)
all_aae['cleaned_text']
